In [1]:
from cartoframes.auth import set_default_credentials
from cartoframes import read_carto, to_carto
import geopandas as gpd
import pandas as pd
import os
import logging 
from shapely import geometry
import requests
import re
from bs4 import BeautifulSoup
import glob
from zipfile import ZipFile
import shutil

NumExpr defaulting to 8 threads.


In [6]:
# set up logging

# get top-level logger object
logger = logging.getLogger()
for handler in logger.handlers: logger.removeHandler(handler)
# manually set level 
logger.setLevel(logging.INFO)
# print to console
console = logging.StreamHandler()
logger.addHandler(console)
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

In [3]:
ebsa = '/home/rthoms/Github/resource-watch/wri-projects/ocean-watch/processing-scripts/biodiversity-protection/EBSA/dissolved_ebsa.shp'
ebsa_gdf = gpd.read_file(ebsa)

In [9]:
print(ebsa_gdf.head())

                                                NAME  \
0                   "Ilha s Tinhosa" (Ilha Principe)   
1  2les des Sept Frbres et Godorya (Seven Brother...   
2                          Agulhas Bank Nursery Area   
3                                      Agulhas Front   
4                        Agulhas slope and seamounts   

                                          Workshop  EBSA_ID GLOBAL_ID  \
0                              South East Atlantic       26    SEA_26   
1  North-West Indian Ocean and Adjacent Gulf Areas       20   NWIO_20   
2                            Southern Indian Ocean        1     SIO_1   
3                            Southern Indian Ocean       11    SIO_11   
4                            Southern Indian Ocean        2     SIO_2   

                                            geometry  
0  POLYGON ((7.26433 1.44636, 7.28321 1.44270, 7....  
1  MULTIPOLYGON (((43.32699 12.47501, 43.45137 12...  
2  POLYGON ((22.22186 -34.26398, 22.22942 -35.104...  
3  P

In [4]:
mpa = '/home/rthoms/Github/resource-watch/wri-projects/ocean-watch/processing-scripts/biodiversity-protection/MPAs/valid_mpa.shp'
mpa_gdf = gpd.read_file(mpa)

In [14]:
print(mpa_gdf.head())

   WDPAID WDPA_PID PA_DEF                                  NAME  \
0     1.0        1      1  Diamond Reef and Salt Fish Tail Reef   
1     2.0        2      1                         Palaster Reef   
2    27.0       27      1                             Folkstone   
3    46.0       46      1      Reserva Biológica Atol Das Rocas   
4    57.0       57      1        Parque Nacional Do Cabo Orange   

                          ORIG_NAME              DESIG           DESIG_ENG  \
0                      Diamond Reef     Marine Reserve      Marine Reserve   
1                     Palaster Reef     Marine Reserve      Marine Reserve   
2                         Folkstone     Marine Reserve      Marine Reserve   
3  Reserva Biológica Atol Das Rocas  Reserva Biológica  Biological Reserve   
4    Parque Nacional Do Cabo Orange             Parque                Park   

  DESIG_TYPE IUCN_CAT        INT_CRIT  ...     MANG_PLAN           VERIF  \
0   National       Ia  Not Applicable  ...  Not Repo

In [7]:
# authenticate carto account  
CARTO_USER = os.getenv('CARTO_WRI_RW_USER')
CARTO_KEY = os.getenv('CARTO_WRI_RW_KEY')
set_default_credentials(username=CARTO_USER, base_url="https://{user}.carto.com/".format(user=CARTO_USER),api_key=CARTO_KEY)

logger.info('Pull EEZ boundaries from Carto table')
#read in the GADM country boundaries used in the OW geostore
EEZ_gdf = read_carto("SELECT * FROM com_011_3_maritime_boundaries_exclusive_economic_zones WHERE iso_ter1 = 'SWE'")


Pull EEZ boundaries from Carto table


In [8]:
mpa_join = gpd.sjoin(ebsa_gdf, mpa_gdf, how = 'right')


In [11]:
print(len(mpa_join.index))

17937


In [10]:
mpa_subset = mpa_join.loc[~mpa_join['EBSA_ID'].isnull()]

In [12]:
print(len(mpa_subset.index))

4020


In [15]:
flat_mpa_subset = mpa_subset.dissolve('WDPA_PID')

In [16]:
print(len(flat_mpa_subset.index))

3364


In [17]:
flat_mpa_subset.to_file('flat_mpa_subset.shp',driver='ESRI Shapefile')

Value 555544090 of field WDPAID of feature 1189 not successfully written. Possibly due to too larger number with respect to field width
Value 555511956 of field WDPAID of feature 1190 not successfully written. Possibly due to too larger number with respect to field width
Value 555511959 of field WDPAID of feature 1191 not successfully written. Possibly due to too larger number with respect to field width
Value 555511961 of field WDPAID of feature 1192 not successfully written. Possibly due to too larger number with respect to field width
Value 555511964 of field WDPAID of feature 1193 not successfully written. Possibly due to too larger number with respect to field width
Value 555511967 of field WDPAID of feature 1194 not successfully written. Possibly due to too larger number with respect to field width
Value 555511973 of field WDPAID of feature 1195 not successfully written. Possibly due to too larger number with respect to field width
Value 555511976 of field WDPAID of feature 1196 

In [ ]:
prot_ebsa = gpd.overlay(flat_mpa_subset, ebsa_gdf, how = 'intersection')